In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt



# Carregar dados
df = pd.read_csv("C:/Users/rgs12/Downloads/productivity+prediction+of+garment+employees/garments_worker_productivity.csv")

# informações básicas e primeiras linhas
df.info()
print(df.head())

# Verificar e ver valores ausentes
missing_counts = df.isnull().sum()
missing_percent = (missing_counts / len(df)) * 100
print("Valores em falta por coluna:\n", missing_counts)
print("\nPercentual de valores em falta por coluna:\n", missing_percent)

# Imputar valores ausentes
median_wip_department = df.groupby('department')['wip'].transform('median')
df['wip'] = df['wip'].fillna(median_wip_department).fillna(df['wip'].median())

# Garantir que não há valores ausentes
assert df['wip'].isnull().sum() == 0, "Ainda existem valores faltantes."
print("\nImputação concluída. Não há mais valores ausentes.")

#Normalização/Padrão
#Selecionamos colunas numéricas (exceto target) para padronização.
numeric_cols = df.select_dtypes(include=['float64','int64']).columns.drop('actual_productivity')
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
# Verificar médias e desvios
print(df[numeric_cols].agg(['mean','std']))

#Aplicação de Modelos de Aprendizagem Automática
#Converter `date` para datetime e extrair `year`, `month`, `day_num`.
#Codificar dummies para `quarter`, `department`, `day`.
#%%
df['date'] = pd.to_datetime(df['date'], dayfirst=False)
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day_num'] = df['date'].dt.day

cat_cols = ['quarter','department','day']
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Preparar dataframe de modelagem (remover colunas irrelevantes)
df_model = df.drop(['date'], axis=1)


# Divisão Treino/Teste
X = df_model.drop('actual_productivity', axis=1)
y = df_model['actual_productivity']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"Treino: {X_train.shape}, Teste: {X_test.shape}")

#Regressão Linear
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)

mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
print(f"Linear Regression -> MSE: {mse_lr:.4f}, R2: {r2_lr:.4f}")

# Scatter plot: previsto vs real
plt.figure()
plt.scatter(y_test, y_pred_lr, alpha=0.6)
plt.xlabel('Productivity Real')
plt.ylabel('Productivity Prevista')
plt.title('Linear Regression: Real vs Previsto')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.show()

#Modelo Avançado: Random Forest + GridSearch
rf = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [50,100],
    'max_depth': [None,10,20],
    'min_samples_split': [2,5]
}
grid = GridSearchCV(rf, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid.fit(X_train, y_train)
best_rf = grid.best_estimator_
print(f"Melhores parâmetros RF: {grid.best_params_}")

y_pred_rf = best_rf.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print(f"Random Forest -> MSE: {mse_rf:.4f}, R2: {r2_rf:.4f}")


#histogramas e mapa de calor?
sns.kdeplot(df['over_time'], fill=True, color='#2ecc71')  
plt.title('Densidade de Horas Extras por Funcionário')  
plt.xlabel('Horas Extras') 

for col in numeric_cols:
    plt.figure()
    plt.hist(df[col], bins=30)
    plt.title(f"Histograma de {col}")
    plt.xlabel(col)
    plt.ylabel('Frequência')
    plt.show()


 

# Correlação
df_corr = df[numeric_cols.tolist() + ['actual_productivity']].corr()
plt.figure(figsize=(10,8))
plt.matshow(df_corr, fignum=1)
plt.title('Mapa de Calor das Correlações', pad=20)
plt.xticks(range(len(df_corr.columns)), df_corr.columns, rotation=90)
plt.yticks(range(len(df_corr.columns)), df_corr.columns)
plt.colorbar()
plt.show()

# Boxplots das variáveis numéricas?
for col in numeric_cols:
    plt.figure()
    plt.boxplot(df[col], vert=False)
    plt.title(f"Boxplot de {col}")
    plt.xlabel(col)
    plt.show()